### PYTHON LIBRARY SETUP

In [ ]:
# Import the ***TextBlob*** class for sentiment analysis and define a function to extract polarity and subjectivity from text, as specified in the instructions.
!pip install textblob


In [ ]:
import kagglehub
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns
# Import the ***nltk*** library and download the necessary 'stopwords' and 'punkt' corpora for text preprocessing, as specified in the instructions.
# This ensures that these resources are available for tokenization and stop word removal.
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
from textblob import TextBlob

### DOWNLOAD AND LOAD DATASET

In [ ]:
# Download Dataset
path_fake_real = kagglehub.dataset_download(
    "clmentbisaillon/fake-and-real-news-dataset"
)
print("fake/real path:", path_fake_real)

path_ai1 = kagglehub.dataset_download(
    "walidbenaouda/ai-isot-dataset"
)
path_ai2 = kagglehub.dataset_download(
    "atharvasoundankar/gen-ai-misinformation-detection-datase-20242025"
)

In [ ]:
# Load the Fake and True CSV Datasets
fake = pd.read_csv(os.path.join(path_fake_real, "Fake.csv")) #cvs for fake
true = pd.read_csv(os.path.join(path_fake_real, "True.csv")) #cvs for true

# Load AI Datasets
ai_isot = pd.read_csv(os.path.join(path_ai1, "AI-ISOT dataset.csv"))
ai_gen = pd.read_csv(os.path.join(path_ai2, "generative_ai_misinformation_dataset.csv"))

In [ ]:
print(f"-----------------------FAKE.CSV---------------------{fake.head()}\n") #shows headers
print(f"-----------------------TRUE.CSV---------------------{true.head()}\n") #shows headers
print(f"-----------------------AI_1.CSV----------------------{ai_isot.head()}\n")   #shows headers
print(f"-----------------------AI_2.CSV----------------------{ai_gen.head()}\n")   #shows headers

### TEXT CLEANING

#### Data Cleaning Function

In [ ]:
# Cleaning text in the dataset
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r"http\S+|www.\S+", "", text) # Gets rid of urls
    text = re.sub(r"<.*?>", "", text) # Gets rid of html
    text = re.sub(r"[^a-zA-Z0-9.,!?'’\s]", " ", text) # Makes sure punctuation still there
    text = re.sub(r"\s+", " ", text).strip() # Fixes spaces
    return text

#### Clean Fake vs True Dataset

In [ ]:
# Clean real vs fake dataset
fake["is_real"] = 0 # Gives fake news a label
true["is_real"] = 1 # Gives true news a label

df_rf = pd.concat([fake, true], ignore_index=True) # Combines datasets

# Makes text in one place
df_rf["text"] = (
    df_rf["title"].fillna("") + " " +
    df_rf["text"].fillna("")
).str.strip()

# Gets rid of duplicates and empty text
df_rf = df_rf.drop_duplicates(subset=["text"])
df_rf = df_rf.dropna(subset=["text"])
df_rf = df_rf[df_rf["text"].str.strip() != ""]
# Cleans the text
df_rf["text"] = df_rf["text"].apply(clean_text)

# Picks the necessary columns and saves the cleaned file
df_rf_clean = df_rf[["text", "is_real"]]
df_rf_clean["source"] = "FAKE-REAL"
df_rf_clean.to_csv("clean_real_fake.csv", index=False)

# Show it was saved and shows portion of cleaned data set
print("saved: clean_real_fake.csv")
print(df_rf_clean.head())

#### Clean AI vs Human Dataset

##### AI-Dataset 1 - AI-ISOT Dataset

In [ ]:
print(ai_isot.columns)

In [ ]:
ai_isot_long = []

# Human-written REAL news : human, real
for x in ai_isot["Real News"].dropna():
    ai_isot_long.append({"text": x, "is_ai": 0, "is_real": 1})

# Human-written FAKE news : human, fake
for x in ai_isot["Fake News"].dropna():
    ai_isot_long.append({"text": x, "is_ai": 0, "is_real": 0})

# AI-generated Fake News : ai, fake
for x in ai_isot["AI-generated Fake News"].dropna():
    ai_isot_long.append({"text": x, "is_ai": 1, "is_real": 0})

ai_isot_df = pd.DataFrame(ai_isot_long)

ai_isot_df["text"] = ai_isot_df["text"].apply(clean_text)
ai_isot_df["source"] = "AI-ISOT"

ai_isot_df.to_csv("clean_ai_isot.csv", index=False)
print("Saved: clean_ai_isot.csv")

##### AI-Dataset 2 - AI_Gen Dataset

In [ ]:
print(ai_gen.columns)

In [ ]:
# Keep only needed columns
ai_gen = ai_gen[[
    "text", "is_misinformation", "model_signature",
    "date", "month", "country", "platform"
]]

# Drop rows with missing text
ai_gen = ai_gen.dropna(subset=["text"])
ai_gen = ai_gen[ai_gen["text"].str.strip() != ""]

# Clean text
ai_gen["text"] = ai_gen["text"].apply(clean_text)

# Convert model_signature to AI/Human label
# 1 = AI-generated, 0 = Human-written
ai_gen = ai_gen[ai_gen["model_signature"].isin(["GPT-like", "human"])]
ai_gen["is_ai"] = ai_gen["model_signature"].apply(
    lambda x: 1 if x == "GPT-like" else 0
)

# Convert misinformation column to binary label
ai_gen["is_real"] = ai_gen["is_misinformation"].apply(
    lambda x: 0 if x == 1 else 1
)

# Final cleaned AI-gen dataset
ai_gen_clean = ai_gen[[
    "text", "is_real", "is_ai", "date", "month", "country", "platform"
]]

ai_gen_clean.to_csv("clean_ai_gen.csv", index=False)
print("Saved: clean_ai_gen.csv")

#### Combined Master Datasets

In [ ]:
combined = pd.concat([df_rf_clean, ai_isot_df, ai_gen], ignore_index=True)

combined = combined.drop_duplicates(subset=["text"])
combined = combined[combined["text"].str.strip() != ""]

combined.to_csv("combined_master_dataset.csv", index=False)
print("Saved: combined_master_dataset.csv")

print("Rows in final dataset:", len(combined))

#### Combine News Datasets Only for EDA:
* df_rf_clean
* ai_isot_df

In [ ]:
combined_news = pd.concat([df_rf_clean, ai_isot_df], ignore_index=True)

combined_news = combined.drop_duplicates(subset=["text"])
combined_news = combined[combined["text"].str.strip() != ""]

combined_news.to_csv("combined_news_dataset.csv", index=False)
print("Saved: combined_news_dataset.csv")

print("Rows in Combined News Dataset:", len(combined_news))

In [ ]:
combined_news.head()

### EDA

Perform an initial data overview of the ***combined_news*** DataFrame by displaying its first few rows, shape, and general information, then analyze and visualize the distribution of the ***is_real*** and ***source*** columns using count plots. Afterwards, handle ***NaN*** values in the ***is_ai*** column by replacing them with -1, and then analyze and visualize its distribution using a count plot. Finally, summarize the key findings from these initial checks.

#### 1. Initial Data Overview








In [ ]:
print("First 5 rows of combined_news DataFrame:")
print(combined_news.head())

print("\nShape of combined_news DataFrame:")
print(combined_news.shape)

print("\nGeneral information about combined_news DataFrame:")
combined_news.info()

#### 2. Analyze and visualize the distribution of the ***is_real*** and ***source*** columns using count plots.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
color = ['mediumpurple','gold']
plt.figure(figsize=(7, 5))
sns.countplot(data=combined_news, x='is_real', hue='is_real', palette=color, legend = False)
plt.title('Distribution of is_real (0=Fake, 1=Real)')
plt.xlabel('Is Real')
plt.ylabel('Count')
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
sns.countplot(data=combined_news, x='source', hue='source', palette=color, legend = False)
plt.title('Distribution of Source')
plt.xlabel('Source')
plt.ylabel('Count')
plt.show()

#### 3. Handle NaN values in the ***is_ai*** column:


Handle `NaN` values in the `is_ai` column by replacing them with -1, and then analyze and visualize its distribution using a count plot.

In [ ]:
combined_news['is_ai'] = combined_news['is_ai'].fillna(-1)

print("Combined_news DataFrame after handling NaN values in 'is_ai':")
print(combined_news.head())

In [ ]:
plt.figure(figsize=(7, 5))
sns.countplot(data=combined_news, x='is_ai', hue='is_ai', palette=color, legend = False)
plt.title('Distribution of is_ai (0=Human, 1=AI, -1=Not applicable)')
plt.xlabel('Is AI')
plt.ylabel('Count')
plt.show()

#### 4. Summary of Key Findings

**4.1. `combined_news` DataFrame Structure:**
*   The DataFrame `combined_news` contains 39,972 entries and 4 columns: `text`, `is_real`, `source`, and `is_ai`.
*   The `text`, `is_real`, and `source` columns are fully populated, with no missing values.
*   The `is_ai` column initially had a large number of missing (NaN) values, as indicated by only 895 non-null entries before handling.

**4.2. Distribution of `is_real`:**
*   The count plot for `is_real` shows a fairly balanced distribution between real (1) and fake (0) news entries, which is good for training classification models.

**4.3. Distribution of `source`:**
*   The `source` column is predominantly composed of entries from "FAKE-REAL", with a much smaller proportion from "AI-ISOT". This indicates that the dataset is heavily skewed towards traditional fake/real news rather than AI-specific news sources.

**4.4. Distribution of `is_ai` after handling NaNs:**
*   After replacing `NaN` values with -1, the count plot for `is_ai` clearly shows three categories:
    *   `-1` (Not applicable): This is the largest category, representing entries where AI attribution was not originally provided (mostly from the FAKE-REAL dataset).
    *   `0` (Human-written): A small number of entries are identified as human-written.
    *   `1` (AI-generated): An even smaller number of entries are identified as AI-generated.
*   This highlights that the `is_ai` column is largely sparse and mainly applicable to a specific subset of the `combined_news` data (i.e., from the AI-ISOT dataset).

#### 5. Text Length and Word Count Analysis


Calculate the number of characters and words for each text entry in the `combined_news` DataFrame. Visualize the distributions of these metrics using histograms or density plots, and compare the average/median text lengths between real (is_real=1) and fake (is_real=0) news.


First, we will calculate the number of characters for each text entry and store it in a new column ***char_count*** in the ***combined_news*** DataFrame.



In [ ]:
combined_news['char_count'] = combined_news['text'].str.len()
print("Added 'char_count' column to combined_news DataFrame.")
print(combined_news.head())

Calculate the number of words for each text entry and store it in a new column ***word_count***.



In [ ]:
combined_news['word_count'] = combined_news['text'].apply(lambda x: len(str(x).split()))
print("Added 'word_count' column to combined_news DataFrame.")
print(combined_news.head())

Visualize the distribution of character count:



In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(combined_news['char_count'], bins=50, kde=True, color='mediumpurple')
plt.title('Distribution of Character Count')
plt.xlabel('Character Count')
plt.ylabel('Frequency')
plt.show()


Visualize the distribution of the ***word_count***.



In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(combined_news['word_count'], bins=50, kde=True, color='gold')
plt.title('Distribution of Word Count')
plt.xlabel('Word Count')
plt.ylabel('Frequency')
plt.show()

Calculate the average and median character count for real and fake news articles.



In [ ]:
print("\nAverage and Median Character Count for Real vs. Fake News:")
print(combined_news.groupby('is_real')['char_count'].agg(['mean', 'median']))

Calculate the average and median word count for real and fake news articles.


In [ ]:
print("Average and Median Word Count for Real vs. Fake News:")
print(combined_news.groupby('is_real')['word_count'].agg(['mean', 'median']))

Visually compare the ***char_count*** distribution between real and fake news using a box plot.



In [ ]:
plt.figure(figsize=(8, 6))
sns.boxplot(data=combined_news, x='is_real', y='char_count', hue='is_real', palette=['mediumpurple', 'gold'], legend=False)
plt.title('Character Count Distribution by News Type (0=Fake, 1=Real)')
plt.xlabel('News Type')
plt.ylabel('Character Count')
plt.xticks([0, 1], ['Fake News', 'Real News'])
plt.show()

Visually compare the ***word_count*** distribution between real and fake news by creating a box plot.



In [ ]:
plt.figure(figsize=(8, 6))
sns.boxplot(data=combined_news, x='is_real', y='word_count', hue='is_real', palette=['mediumpurple', 'gold'], legend=False)
plt.title('Word Count Distribution by News Type (0=Fake, 1=Real)')
plt.xlabel('News Type')
plt.ylabel('Word Count')
plt.xticks([0, 1], ['Fake News', 'Real News'])
plt.show()

#### Summary of Text Length and Word Count Analysis

**5.1. Character and Word Counts:**
*   New columns ***char_count*** and ***word_count*** were successfully added to the ***combined_news*** DataFrame, providing quantitative metrics for text length.

**5.2. Distribution of Text Lengths (Character and Word Counts):**
*   Both character and word count distributions show a wide range, indicating variability in article lengths within the dataset. Most articles tend to be shorter, with a long tail extending towards very long articles.
*   The distributions appear to be right-skewed, meaning there are many shorter articles and fewer very long ones.

**5.3. Comparison of Real vs. Fake News (Character Count):**
*   **Average Character Count:** Fake news has a slightly higher average character count (approximately 2551) compared to real news (approximately 2424).
*   **Median Character Count:** The medians are very close (2281 for fake, 2265 for real), suggesting that for the majority of articles, the character lengths are quite similar.
*   The box plots for character count indicate that fake news tends to have a slightly wider spread and potentially more outliers on the higher end, suggesting some fake news articles can be considerably longer.

**5.4. Comparison of Real vs. Fake News (Word Count):**
*   **Average Word Count:** Similar to character count, fake news articles have a higher average word count (approximately 434) than real news (approximately 398).
*   **Median Word Count:** The medians are also close (390 for fake, 372 for real).
*   The box plots for word count also show a similar pattern to character count, with fake news potentially having a slightly larger range and more long-form outliers.

**Overall Conclusion:**
While there isn't a drastic difference, fake news articles in this combined dataset tend to be slightly longer on average in terms of both character and word counts compared to real news articles. This could be a subtle distinguishing feature, although the distributions largely overlap.

#### 6. Word Frequency and Vocabulary Insights


Extract the most frequent words from the 'text' column. Visualize the top 20 most common words across the entire dataset, and then separately for real and fake news using bar charts or word clouds to identify distinctive vocabulary patterns.


In [ ]:
nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)
print("NLTK stopwords and punkt corpora downloaded.")

Create a text preprocessing function that converts text to lowercase, tokenizes it, removes stopwords, and filters out non-alphabetic tokens, as described in the instructions.



In [ ]:
stop_words = set(stopwords.words('english'))
def preprocess_text(text):
    text = str(text).lower()  # Convert to string and lowercase
    tokens = word_tokenize(text) # Tokenize the text
    # Remove stop words and non-alphabetic tokens
    filtered_tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return filtered_tokens

print("Text preprocessing function 'preprocess_text' created.")

Now that the preprocessing function is defined, let's apply it to the 'text' column of the ***combined_news*** DataFrame to tokenize and clean the text, storing the result in a new column called 'processed_text'.



In [ ]:
nltk.download('punkt_tab', quiet=True)
print("NLTK 'punkt_tab' corpus downloaded.")

In [ ]:
combined_news['processed_text'] = combined_news['text'].apply(preprocess_text)
print("Applied 'preprocess_text' to the 'text' column.")
print(combined_news[['text', 'processed_text']].head())

Now that the text is preprocessed and stored as lists of words in the 'processed_text' column, let's combine all these lists into a single flat list to prepare for counting the most frequent words across the entire dataset.



In [ ]:
all_words = [word for sublist in combined_news['processed_text'] for word in sublist]
print(f"Total words in the dataset after preprocessing: {len(all_words)}")

Use the ***collections.Counter*** to find the 20 most frequent words, which is a required step for visualizing the top common words across the entire dataset.



In [ ]:
word_freq = Counter(all_words)
top_20_words = word_freq.most_common(20)
print("Top 20 most common words across the entire dataset:")
print(top_20_words)

Extract the words and their corresponding frequencies from the ***top_20_words*** list and then use ***matplotlib.pyplot*** and ***seaborn*** to create a bar chart.



In [ ]:
words = [word for word, count in top_20_words]
counts = [count for word, count in top_20_words]

plt.figure(figsize=(12, 7))
sns.barplot(x=words, y=counts, hue=words, palette='viridis', legend=False)
plt.title('Top 20 Most Common Words Across All News')
plt.xlabel('Words')
plt.ylabel('Frequency')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()
print("Bar plot of top 20 most common words across the entire dataset displayed.")

Filter the ***combined_news*** DataFrame to create a separate DataFrame for real news (where ***is_real*** is 1) to analyze its unique vocabulary patterns.



In [ ]:
real_news_df = combined_news[combined_news['is_real'] == 1]
print("Filtered real news DataFrame created.")
print(f"Number of real news entries: {len(real_news_df)}")
print(real_news_df.head())

Combine all the preprocessed words from the ***processed_text*** column into a single list, which is a necessary step before counting the most frequent words.



In [ ]:
real_news_words = [word for sublist in real_news_df['processed_text'] for word in sublist]
print(f"Total words in real news dataset: {len(real_news_words)}")

Use ***Collections.Counter*** to find the 20 most frequent words specifically for real news.



In [ ]:
real_word_freq = Counter(real_news_words)
top_20_real_words = real_word_freq.most_common(20)
print("Top 20 most common words in real news:")
print(top_20_real_words)

To visualize the top 20 most common words in real news, I need to extract the words and their corresponding frequencies from the ***top_20_real_words*** list and then create a bar chart.



In [ ]:
words_real = [word for word, count in top_20_real_words]
counts_real = [count for word, count in top_20_real_words]

plt.figure(figsize=(12, 7))
sns.barplot(x=words_real, y=counts_real, hue=words_real, palette='viridis', legend=False)
plt.title('Top 20 Most Common Words in Real News')
plt.xlabel('Words')
plt.ylabel('Frequency')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()
print("Bar plot of top 20 most common words in real news displayed.")

Filter the ***combined_news*** DataFrame to create a separate DataFrame for fake news (where ***is_real*** is 0), as specified in the instructions, to analyze its unique vocabulary patterns.



In [ ]:
fake_news_df = combined_news[combined_news['is_real'] == 0]
print("Filtered fake news DataFrame created.")
print(f"Number of fake news entries: {len(fake_news_df)}")
print(fake_news_df.head())

In [ ]:
fake_news_words = [word for sublist in fake_news_df['processed_text'] for word in sublist]
print(f"Total words in fake news dataset: {len(fake_news_words)}")

Now that all processed words from fake news are combined into a single list, let's find the 20 most frequent words specifically for fake news, as specified in the instructions.



In [ ]:
fake_word_freq = Counter(fake_news_words)
top_20_fake_words = fake_word_freq.most_common(20)
print("Top 20 most common words in fake news:")
print(top_20_fake_words)

#### 7. Sentiment Analysis

Perform sentiment analysis on the 'text' column to derive polarity (emotional tone, e.g., positive, negative, neutral) and subjectivity scores. Add these as new columns to the DataFrame and visualize their distributions. Compare the sentiment scores between real and fake news to identify potential differences in emotional content.


Define the ***get_sentiment*** function as specified in the instructions to extract polarity and subjectivity scores.



In [ ]:
def get_sentiment(text):
    blob = TextBlob(str(text))
    return blob.sentiment.polarity, blob.sentiment.subjectivity

print("Imported TextBlob and defined 'get_sentiment' function.")

Now that the ***get_sentiment*** function is defined, let's apply it to the 'text' column of the ***combined_news*** DataFrame to calculate the polarity and subjectivity scores for each text entry. The results will be stored in new columns named 'polarity' and 'subjectivity' respectively.



In [ ]:
combined_news[['polarity', 'subjectivity']] = combined_news['text'].apply(lambda x: pd.Series(get_sentiment(x)))
print("Added 'polarity' and 'subjectivity' columns to combined_news DataFrame.")
print(combined_news.head())

Visualize its distribution using a histogram or density plot to understand the overall emotional tone of the news articles.



In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(combined_news['polarity'], bins=50, kde=True, color='mediumpurple')
plt.title('Distribution of Polarity Scores')
plt.xlabel('Polarity Score')
plt.ylabel('Frequency')
plt.show()

Now that the 'subjectivity' column has been added to the DataFrame, let's visualize its distribution using a histogram or density plot to understand the degree of factual information versus personal opinions in the news articles.



In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(combined_news['subjectivity'], bins=50, kde=True, color='gold')
plt.title('Distribution of Subjectivity Scores')
plt.xlabel('Subjectivity Score')
plt.ylabel('Frequency')
plt.show()

Compare the average and median polarity scores between real and fake news articles to identify potential differences in emotional tone.



In [ ]:
print("\nAverage and Median Polarity Scores for Real vs. Fake News:")
print(combined_news.groupby('is_real')['polarity'].agg(['mean', 'median']))

Compare the average and median subjectivity scores between real and fake news articles to identify potential differences in the degree of factual information versus personal opinions.



In [ ]:
print("\nAverage and Median Subjectivity Scores for Real vs. Fake News:")
print(combined_news.groupby('is_real')['subjectivity'].agg(['mean', 'median']))

#### Summary of Sentiment Analysis

**1. Polarity Scores:**
*   The distribution of polarity scores across all news articles is centered around zero, indicating a generally neutral emotional tone, with a slight tendency towards positive sentiment.
*   There is a relatively wide spread, suggesting a variety of emotional tones present in the dataset.
*   **Comparison (Real vs. Fake News):** Both real and fake news articles exhibit very similar average and median polarity scores, indicating that, on average, their emotional tones are quite close. Fake news has a slightly higher average polarity (approx. 0.0598) than real news (approx. 0.0573), but the difference is marginal.

**2. Subjectivity Scores:**
*   The distribution of subjectivity scores shows a considerable number of articles leaning towards factual (lower subjectivity) content, but also a significant portion with higher subjectivity, indicating opinion-based writing.
*   **Comparison (Real vs. Fake News):** A more notable difference is observed in subjectivity scores. Fake news articles have a significantly higher average subjectivity (approx. 0.4549) and median subjectivity (approx. 0.4552) compared to real news articles (average approx. 0.3624, median approx. 0.3676). This suggests that fake news tends to contain more personal opinions and less factual reporting than real news.

**Overall Conclusion:**
Sentiment analysis reveals a clearer distinction in subjectivity than in polarity. While both real and fake news exhibit similar emotional tones, fake news articles are generally more subjective, implying a higher presence of opinions, beliefs, and personal feelings compared to the more objective nature of real news. This difference in subjectivity could be a valuable feature for distinguishing between the two news types.

#### Text Analysis Summary:




##### Key Findings:

*   **Text Length Comparison:** Fake news articles tend to be slightly longer than real news articles on average. The average character count for fake news was approximately 2551, while for real news it was around 2424. Similarly, the average word count for fake news was about 434, compared to approximately 398 for real news. However, median lengths were very similar, and distributions largely overlapped, suggesting that while there's a slight tendency, length isn't a strong discriminator on its own.
*   **Word Frequency Patterns:**
    *   "Trump" is a highly frequent word across all news types, appearing as the most common word in fake and overall news, and the second most common in real news.
    *   Real news frequently features journalistic terms like "said" and source attributions like "reuters" (28,861 occurrences in real news).
    *   Fake news frequently uses words like "people," "president," "one," "donald," "like," "obama," "clinton," "video," and "hillary," suggesting a focus on specific figures, personal opinions, and potentially sensational content.
*   **Sentiment Polarity:** Both real and fake news exhibit very similar emotional tones, with average polarity scores being approximately 0.0573 for real news and 0.0598 for fake news. The distributions for polarity are centered around zero, indicating a generally neutral to slightly positive emotional tone for both categories.
*   **Sentiment Subjectivity:** A significant difference was observed in subjectivity. Fake news articles are notably more subjective, with an average subjectivity score of approximately 0.4549 and a median of 0.4552. In contrast, real news articles are more objective, with an average subjectivity score of approximately 0.3624 and a median of 0.3676. This indicates that fake news often contains more opinions and beliefs.

### MODELS and ANALYSIS

#### Long Short-Term Memory (LSTM)

#### BERT Modeling and Integration Part 1:

This section builds on existing EDA work and uses the **combined_news** dataset
we have already prepared, which includes:
- Real/Fake news (df_rf_clean)
- AI-ISOT news dataset (ai_isot_df)

MEMORY OPTIMIZATION STRATEGIES:
- Using DistilBERT (lighter than BERT)
- Gradient accumulation for effective larger batches
- Reduced max sequence length to 256 tokens
- Memory cleanup between stages

In [ ]:
# ============================================================================
# CHECK DATA SIZE AND SAMPLE IF NEEDED
# ============================================================================

print(f"Original combined_news size: {len(combined_news)} samples")

# IMPORTANT: Sample the data to speed up training
# Adjust SAMPLE_SIZE:
# - 5000 = ~30-45 min total training time (RECOMMENDED for testing)
# - 10000 = ~1-2 hours
# - 20000 = ~3-4 hours
# - None = use all data (8+ hours)

SAMPLE_SIZE = 5000  # Start small to test!

if SAMPLE_SIZE and len(combined_news) > SAMPLE_SIZE:
    print(f"⚠️  Sampling {SAMPLE_SIZE} samples for faster training...")
    combined_news_sampled = combined_news.sample(n=SAMPLE_SIZE, random_state=42).reset_index(drop=True)
    print(f"✓ Using {len(combined_news_sampled)} samples")

    # Replace combined_news with the sampled version
    combined_news = combined_news_sampled
else:
    print(f"✓ Using all {len(combined_news)} samples")

print(f"\nClass distribution:")
print(combined_news['is_real'].value_counts())

In [ ]:
# ============================================================================
# BERT MODELING SETUP - OPTIMIZED FOR GOOGLE COLAB
# ============================================================================

# Install NEW required libraries only (pandas, numpy, matplotlib already imported)
!pip install transformers torch scikit-learn bertviz

# Import garbage collection for memory management
import gc

# Import NEW libraries for BERT (not already in your notebook)
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    DistilBertTokenizer, DistilBertForSequenceClassification,
    get_linear_schedule_with_warmup
)
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tqdm import tqdm

# Check if GPU is available and set device accordingly
# GPU acceleration significantly speeds up training (10-100x faster)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    # Clear GPU cache
    torch.cuda.empty_cache()
    print(f"GPU Memory Available: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

# ============================================================================
# CUSTOM DATASET CLASS
# ============================================================================

"""
Creating a custom PyTorch Dataset class for our news text data.
This class handles:
- Text tokenization (converting words to numbers BERT understands)
- Padding/truncation to fixed length
- Creating attention masks (tells model which tokens are real vs padding)
- Returning data in the format PyTorch expects
"""

class NewsDataset(Dataset):

    def __init__(self, texts, labels, tokenizer, max_length=256):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        """
        Gets a single sample from the dataset.
        This is called by DataLoader during training.

        Returns a dictionary containing:
        - input_ids: Token IDs for BERT
        - attention_mask: Mask indicating real tokens vs padding
        - labels: The true label for this sample
        """
        text = str(self.texts[idx])
        label = self.labels[idx]

        # Tokenize the text using BERT tokenizer
        # This converts words to numerical IDs that BERT understands
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,      # Adds [CLS] and [SEP] tokens
            max_length=self.max_length,    # Truncate longer sequences
            padding='max_length',          # Pad shorter sequences
            truncation=True,               # Truncate if too long
            return_attention_mask=True,    # Create attention mask
            return_tensors='pt'            # Return PyTorch tensors
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# ============================================================================
# STAGE 1: REAL/FAKE CLASSIFIER
# ============================================================================

"""
STAGE 1 OVERVIEW:
This is our first classifier that determines if a news article is REAL or FAKE.

Why two stages?
- Stage 1: Focuses on authenticity (is this real news?)
- Stage 2: Focuses on authorship (was this written by AI?)

This hierarchical approach allows each model to specialize on its specific task.

We're using the combined_news dataset that includes:
- Real and Fake news articles
- News with AI/Human labels from AI-ISOT dataset
"""

print("\n" + "="*80)
print("STAGE 1: Training REAL/FAKE News Classifier")
print("="*80)

# Use the combined_news DataFrame that's already in your notebook
# This combines df_rf_clean and ai_isot_df
print(f"Using combined_news DataFrame with {len(combined_news)} samples")
print(f"Class distribution:\n{combined_news['is_real'].value_counts()}")

# Prepare data for Stage 1 (Real vs Fake classification)
# We only need the 'text' and 'is_real' columns
df_stage1 = combined_news[['text', 'is_real']].dropna().copy()

print(f"\nAfter removing NaN values: {len(df_stage1)} samples")

# Split data into train (70%), validation (15%), and test (15%)
# Stratify ensures balanced class distribution in all splits
X_train_s1, X_temp_s1, y_train_s1, y_temp_s1 = train_test_split(
    df_stage1['text'].values,
    df_stage1['is_real'].values,
    test_size=0.3,           # 30% for val + test
    random_state=42,         # For reproducibility
    stratify=df_stage1['is_real'].values  # Balanced split
)

# Split the 30% into validation and test (15% each)
X_val_s1, X_test_s1, y_val_s1, y_test_s1 = train_test_split(
    X_temp_s1, y_temp_s1,
    test_size=0.5,           # 50% of 30% = 15% of total
    random_state=42,
    stratify=y_temp_s1
)

print(f"\nData Split:")
print(f"  Train: {len(X_train_s1)} samples")
print(f"  Validation: {len(X_val_s1)} samples")
print(f"  Test: {len(X_test_s1)} samples")

# Initialize the BERT tokenizer for Stage 1
# This converts text to tokens that DistilBERT can process
tokenizer_s1 = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Create PyTorch datasets for train, validation, and test
# These wrap our data and handle tokenization
train_dataset_s1 = NewsDataset(X_train_s1, y_train_s1, tokenizer_s1)
val_dataset_s1 = NewsDataset(X_val_s1, y_val_s1, tokenizer_s1)
test_dataset_s1 = NewsDataset(X_test_s1, y_test_s1, tokenizer_s1)

# Create DataLoaders
# DataLoaders handle batching, shuffling, and parallel data loading
BATCH_SIZE = 8  # Small batch size for memory optimization
train_loader_s1 = DataLoader(train_dataset_s1, batch_size=BATCH_SIZE, shuffle=True)
val_loader_s1 = DataLoader(val_dataset_s1, batch_size=BATCH_SIZE)
test_loader_s1 = DataLoader(test_dataset_s1, batch_size=BATCH_SIZE)

print(f"\nBatch Configuration:")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Training batches: {len(train_loader_s1)}")
print(f"  Validation batches: {len(val_loader_s1)}")
print(f"  Test batches: {len(test_loader_s1)}")

# Initialize the DistilBERT model for sequence classification
# We're using a pre-trained model and fine-tuning it on our data
model_s1 = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',  # Pre-trained model name
    num_labels=2                 # Binary classification (Real vs Fake)
).to(device)                     # Move model to GPU if available

# Training hyperparameters
# These control how the model learns
EPOCHS = 2           # Number of passes through the dataset
LEARNING_RATE = 2e-5 # How big of steps to take during optimization (standard for BERT)
ACCUMULATION_STEPS = 2  # Gradient accumulation for effective batch size of 16

print(f"\nTraining Configuration:")
print(f"  Epochs: {EPOCHS}")
print(f"  Learning Rate: {LEARNING_RATE}")
print(f"  Batch Size: {BATCH_SIZE}")
print(f"  Accumulation Steps: {ACCUMULATION_STEPS}")
print(f"  Effective Batch Size: {BATCH_SIZE * ACCUMULATION_STEPS}")

# Set up optimizer (AdamW is specifically designed for transformers)
optimizer_s1 = AdamW(model_s1.parameters(), lr=LEARNING_RATE)

# Set up learning rate scheduler with warmup
# This gradually increases learning rate at start, then decreases
# Helps with training stability and convergence
total_steps = len(train_loader_s1) * EPOCHS
scheduler_s1 = get_linear_schedule_with_warmup(
    optimizer_s1,
    num_warmup_steps=0,           # No warmup steps
    num_training_steps=total_steps # Total training steps
)

# ============================================================================
# TRAINING FUNCTION
# ============================================================================

"""
Training function that handles one complete pass through the training data.

For each batch:
1. Forward pass: Get predictions from model
2. Calculate loss: How wrong were the predictions?
3. Backward pass: Calculate gradients
4. Optimizer step: Update model weights (with gradient accumulation)
5. Track metrics: Loss and accuracy
"""

def train_epoch(model, dataloader, optimizer, scheduler, device, accumulation_steps=1):
    """Train for one epoch with gradient accumulation for memory efficiency"""
    # Set model to training mode (enables dropout, batch norm updates, etc.)
    model.train()

    # Initialize tracking variables
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0

    # Progress bar for visual feedback during training
    progress_bar = tqdm(dataloader, desc="Training")

    # Iterate through batches
    for batch_idx, batch in enumerate(progress_bar):
        # Move data to GPU
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Forward pass: get model predictions
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        # Extract loss and predictions
        loss = outputs.loss / accumulation_steps  # Normalize loss for accumulation
        logits = outputs.logits  # Raw prediction scores

        # Backward pass: calculate gradients
        loss.backward()

        # Only update weights every accumulation_steps
        if (batch_idx + 1) % accumulation_steps == 0:
            # Clip gradients to prevent exploding gradients problem
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update model weights
            optimizer.step()

            # Update learning rate
            scheduler.step()

            # Zero out gradients for next accumulation
            optimizer.zero_grad()

        # Track metrics
        total_loss += loss.item() * accumulation_steps

        # Convert logits to predictions (highest score wins)
        preds = torch.argmax(logits, dim=1)
        correct_predictions += torch.sum(preds == labels)
        total_predictions += labels.size(0)

        # Update progress bar
        progress_bar.set_postfix({'loss': loss.item() * accumulation_steps})

        # Clear some memory periodically
        del input_ids, attention_mask, labels, outputs, loss, logits
        if batch_idx % 100 == 0 and torch.cuda.is_available():
            torch.cuda.empty_cache()

    # Calculate average loss and accuracy for the epoch
    avg_loss = total_loss / len(dataloader)
    accuracy = correct_predictions.double() / total_predictions

    return avg_loss, accuracy.item()

def eval_model(model, dataloader, device):
    """
    Evaluate model on validation or test set.

    No gradient calculation needed (saves memory and speeds up evaluation).
    Returns predictions, true labels, and confidence scores.
    """
    # Set model to evaluation mode (disables dropout, etc.)
    model.eval()

    total_loss = 0
    predictions = []
    true_labels = []
    confidences = []

    # No gradient calculation during evaluation
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            # Move data to device
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Forward pass only
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )

            loss = outputs.loss
            logits = outputs.logits

            total_loss += loss.item()

            # Convert logits to probabilities using softmax
            probs = torch.softmax(logits, dim=1)

            # Get predicted class and confidence score
            conf, preds = torch.max(probs, dim=1)

            # Store predictions for analysis
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
            confidences.extend(conf.cpu().numpy())

            # Clean up
            del input_ids, attention_mask, labels, outputs, loss, logits, probs

    # Clear cache after evaluation
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    avg_loss = total_loss / len(dataloader)
    accuracy = accuracy_score(true_labels, predictions)

    return avg_loss, accuracy, predictions, true_labels, confidences

# ============================================================================
# TRAIN STAGE 1 MODEL
# ============================================================================

"""
Now we'll train the Stage 1 model through multiple epochs.
Each epoch involves:
1. Training on the training set
2. Evaluating on the validation set
3. Tracking metrics to monitor progress

We save the best model and track learning curves.
"""

print("\n" + "="*80)
print("TRAINING STAGE 1 MODEL")
print("="*80)

# Lists to store metrics for plotting learning curves
train_losses_s1 = []
val_losses_s1 = []
train_accs_s1 = []
val_accs_s1 = []

# Training loop
for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch + 1}/{EPOCHS}")
    print("-" * 60)

    # Train for one epoch with gradient accumulation
    train_loss, train_acc = train_epoch(
        model_s1, train_loader_s1, optimizer_s1, scheduler_s1, device, ACCUMULATION_STEPS
    )

    # Validate after each epoch
    val_loss, val_acc, _, _, _ = eval_model(model_s1, val_loader_s1, device)

    # Store metrics
    train_losses_s1.append(train_loss)
    val_losses_s1.append(val_loss)
    train_accs_s1.append(train_acc)
    val_accs_s1.append(val_acc)

    # Print epoch results
    print(f"\n📊 Epoch {epoch + 1} Results:")
    print(f"   Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"   Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc:.4f}")

    # Memory cleanup after each epoch
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

# Save the trained model for later use
torch.save(model_s1.state_dict(), 'model_stage1_real_fake.pth')
print("\n✅ Stage 1 model saved as 'model_stage1_real_fake.pth'")

# Memory cleanup before evaluation
del train_loader_s1, val_loader_s1
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# ============================================================================
# EVALUATE ON TEST SET
# ============================================================================

"""
Final evaluation on the held-out test set.
This gives us an unbiased estimate of model performance.

We'll generate:
- Overall accuracy
- Classification report (precision, recall, F1 for each class)
- Confusion matrix visualization
"""

print("\n" + "="*80)
print("EVALUATING STAGE 1 ON TEST SET")
print("="*80)

test_loss_s1, test_acc_s1, test_preds_s1, test_labels_s1, test_conf_s1 = eval_model(
    model_s1, test_loader_s1, device
)

print(f"\n📊 Stage 1 Test Results:")
print(f"   Test Loss: {test_loss_s1:.4f}")
print(f"   Test Accuracy: {test_acc_s1:.4f}")
print(f"\n📋 Classification Report:")
print(classification_report(test_labels_s1, test_preds_s1,
                          target_names=['Fake', 'Real']))

# Plot confusion matrix to see where the model makes mistakes
plt.figure(figsize=(8, 6))
cm_s1 = confusion_matrix(test_labels_s1, test_preds_s1)
sns.heatmap(cm_s1, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Fake', 'Real'],
            yticklabels=['Fake', 'Real'])
plt.title('Stage 1: Real/Fake Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.show()

# ============================================================================
# VISUALIZE TRAINING HISTORY
# ============================================================================

"""
Plot learning curves to understand training dynamics:
- Left plot: Loss over epochs (should decrease)
- Right plot: Accuracy over epochs (should increase)

If validation metrics get worse while training improves, that's overfitting.
"""

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Loss curves
ax1.plot(train_losses_s1, label='Train Loss', marker='o', linewidth=2)
ax1.plot(val_losses_s1, label='Val Loss', marker='s', linewidth=2)
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Loss', fontsize=12)
ax1.set_title('Stage 1: Training and Validation Loss', fontsize=14)
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# Accuracy curves
ax2.plot(train_accs_s1, label='Train Accuracy', marker='o', linewidth=2)
ax2.plot(val_accs_s1, label='Val Accuracy', marker='s', linewidth=2)
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Accuracy', fontsize=12)
ax2.set_title('Stage 1: Training and Validation Accuracy', fontsize=14)
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# ============================================================================
# MEMORY CLEANUP BEFORE STAGE 2
# ============================================================================

"""
Clean up memory before moving to Stage 2.
This is crucial for preventing crashes in Google Colab.
"""

print("\n" + "="*80)
print("MEMORY CLEANUP")
print("="*80)

# Clear unnecessary variables
print("Cleaning up Stage 1 variables...")
del train_dataset_s1, val_dataset_s1
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(f"✓ GPU Memory Allocated: {torch.cuda.memory_allocated(0) / 1e9:.2f} GB")
    print(f"✓ GPU Memory Cached: {torch.cuda.memory_reserved(0) / 1e9:.2f} GB")

print("\n✅ Stage 1 complete! Ready for Stage 2.")

##### Stage 1 (Real/Fake Classifier) Phase Summary

This stage focused on building a robust classifier to distinguish between real and fake news articles. It served as the primary filter in our two-stage classification pipeline. The key steps and findings were:

*   **Purpose:** The main objective of Stage 1 was to accurately classify incoming news articles as either 'Real' (1) or 'Fake' (0).
*   **Data Sampling:** To optimize training time and resource utilization, especially within the Colab environment, the combined_news dataset was sampled down to 5000 entries. This allowed for quicker iteration and experimentation while still providing a sufficiently large dataset for meaningful training. The class distribution of the sampled data was verified to ensure balance.
*   **Model Setup:** We utilized DistilBertTokenizer and DistilBertForSequenceClassification from the transformers library. DistilBERT was chosen for its balance of performance and efficiency, being lighter than full BERT models. The tokenizer was initialized with distilbert-base-uncased, and the model was configured for binary classification (num_labels=2) and moved to the available computing device (GPU if available, otherwise CPU).
*   **Data Splitting:** The sampled data (df_stage1) was meticulously split into training (70%), validation (15%), and test (15%) sets using train_test_split. Crucially, stratify was employed to ensure that the proportion of real and fake news was maintained across all splits, preventing class imbalance issues during training and evaluation.
*   **Training Process:** The model was trained over 2 EPOCHS with a LEARNING_RATE of 2e-5. A BATCH_SIZE of 8 was used in conjunction with ACCUMULATION_STEPS = 2, effectively simulating an effective batch size of 16 to manage memory while still achieving good gradient updates. An AdamW optimizer and a linear scheduler with warmup were employed. The custom train_epoch function handled the training loop, including gradient clipping and periodic memory cleanup. The eval_model function performed evaluations on the validation set after each epoch.
*   **Performance on Test Set:** The Stage 1 model demonstrated exceptional performance on the held-out test set:
    *   **Test Accuracy:** 0.9933 (99.33%)
    *   **Classification Report:** Showed high precision, recall, and F1-scores for both 'Fake' and 'Real' classes, indicating robust classification across both categories.
    *   **Confusion Matrix:** Revealed very few misclassifications, with the model accurately identifying the vast majority of real and fake news articles.
*   **Training History Visualization:** Plots of training and validation loss and accuracy curves were generated. These showed a clear decrease in loss and an increase in accuracy for both sets over the epochs, suggesting good learning without significant overfitting.

In [ ]:
# ============================================================================
# STAGE 2: HUMAN/AI CLASSIFIER (STREAMLINED)
# ============================================================================

print("\n" + "="*80)
print("STAGE 2: Training HUMAN/AI Classifier")
print("="*80)

# Memory cleanup
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# Load Stage 2 data - only samples with VALID is_ai labels (0 or 1, not -1)
df_stage2 = combined_news[combined_news['is_ai'].notna()][['text', 'is_ai']].copy()

# CRITICAL FIX: Remove -1 values (which mean "Not applicable")
print(f"Before filtering: {len(df_stage2)} samples")
print(f"is_ai distribution:\n{df_stage2['is_ai'].value_counts()}")

df_stage2 = df_stage2[df_stage2['is_ai'].isin([0, 1])].dropna()

print(f"\nAfter filtering: {len(df_stage2)} samples")
print(f"Class distribution:\n{df_stage2['is_ai'].value_counts()}")

# If there's no data for Stage 2, skip it
if len(df_stage2) == 0:
    print("\n⚠️  No AI/Human labeled data available for Stage 2!")
    print("Skipping Stage 2 training...")

    # Create a dummy classifier that always returns None for Stage 2
    class TwoStageClassifierStage1Only:
        def __init__(self, model_s1, tokenizer_s1, device):
            self.model_s1 = model_s1
            self.tokenizer_s1 = tokenizer_s1
            self.device = device
            self.model_s1.eval()

        def predict(self, text, max_length=256):
            encoding_s1 = self.tokenizer_s1.encode_plus(
                text, add_special_tokens=True, max_length=max_length,
                padding='max_length', truncation=True,
                return_attention_mask=True, return_tensors='pt'
            )

            input_ids_s1 = encoding_s1['input_ids'].to(self.device)
            attention_mask_s1 = encoding_s1['attention_mask'].to(self.device)

            with torch.no_grad():
                outputs_s1 = self.model_s1(input_ids=input_ids_s1, attention_mask=attention_mask_s1)
                probs_s1 = torch.softmax(outputs_s1.logits, dim=1)
                conf_s1, pred_s1 = torch.max(probs_s1, dim=1)

            is_real = pred_s1.item()
            real_confidence = conf_s1.item()

            return {
                'is_real': is_real,
                'real_confidence': real_confidence,
                'real_label': 'Real' if is_real == 1 else 'Fake',
                'is_ai': None,
                'ai_confidence': None,
                'ai_label': 'N/A (Stage 2 not trained)'
            }

        def predict_batch(self, texts):
            return [self.predict(text) for text in texts]

    two_stage_classifier = TwoStageClassifierStage1Only(model_s1, tokenizer_s1, device)
    print("✅ Using Stage 1 only classifier")

else:
    # Continue with normal Stage 2 training...
    # Split data
    X_train_s2, X_temp_s2, y_train_s2, y_temp_s2 = train_test_split(
        df_stage2['text'].values,
        df_stage2['is_ai'].values,
        test_size=0.3,
        random_state=42,
        stratify=df_stage2['is_ai'].values
    )

    X_val_s2, X_test_s2, y_val_s2, y_test_s2 = train_test_split(
        X_temp_s2, y_temp_s2,
        test_size=0.5,
        random_state=42,
        stratify=y_temp_s2
    )

    print(f"Train: {len(X_train_s2)}, Val: {len(X_val_s2)}, Test: {len(X_test_s2)}")

    # Create datasets
    tokenizer_s2 = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
    train_dataset_s2 = NewsDataset(X_train_s2, y_train_s2.astype(int), tokenizer_s2)
    val_dataset_s2 = NewsDataset(X_val_s2, y_val_s2.astype(int), tokenizer_s2)
    test_dataset_s2 = NewsDataset(X_test_s2, y_test_s2.astype(int), tokenizer_s2)

    # Create loaders
    train_loader_s2 = DataLoader(train_dataset_s2, batch_size=BATCH_SIZE, shuffle=True)
    val_loader_s2 = DataLoader(val_dataset_s2, batch_size=BATCH_SIZE)
    test_loader_s2 = DataLoader(test_dataset_s2, batch_size=BATCH_SIZE)

    # Initialize model
    model_s2 = DistilBertForSequenceClassification.from_pretrained(
        'distilbert-base-uncased',
        num_labels=2
    ).to(device)

    optimizer_s2 = AdamW(model_s2.parameters(), lr=LEARNING_RATE)
    total_steps_s2 = len(train_loader_s2) * EPOCHS
    scheduler_s2 = get_linear_schedule_with_warmup(
        optimizer_s2,
        num_warmup_steps=0,
        num_training_steps=total_steps_s2
    )

    # Train Stage 2
    print("\nTraining Stage 2...")
    train_losses_s2 = []
    val_losses_s2 = []
    train_accs_s2 = []
    val_accs_s2 = []

    for epoch in range(EPOCHS):
        print(f"\nEpoch {epoch + 1}/{EPOCHS}")

        train_loss, train_acc = train_epoch(
            model_s2, train_loader_s2, optimizer_s2, scheduler_s2, device, ACCUMULATION_STEPS
        )

        val_loss, val_acc, _, _, _ = eval_model(model_s2, val_loader_s2, device)

        train_losses_s2.append(train_loss)
        val_losses_s2.append(val_loss)
        train_accs_s2.append(train_acc)
        val_accs_s2.append(val_acc)

        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
        print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    # Save model
    torch.save(model_s2.state_dict(), 'model_stage2_human_ai.pth')
    print("\n✅ Stage 2 model saved")

    # Cleanup
    del train_loader_s2, val_loader_s2
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    # Evaluate on test
    print("\nEvaluating Stage 2...")
    test_loss_s2, test_acc_s2, test_preds_s2, test_labels_s2, test_conf_s2 = eval_model(
        model_s2, test_loader_s2, device
    )

    print(f"\nStage 2 Test Accuracy: {test_acc_s2:.4f}")
    print(f"\nClassification Report:")
    print(classification_report(test_labels_s2, test_preds_s2, target_names=['Human', 'AI']))

    # Confusion matrix
    plt.figure(figsize=(8, 6))
    cm_s2 = confusion_matrix(test_labels_s2, test_preds_s2)
    sns.heatmap(cm_s2, annot=True, fmt='d', cmap='Greens',
                xticklabels=['Human', 'AI'],
                yticklabels=['Human', 'AI'])
    plt.title('Stage 2: Human/AI Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.show()

    # ============================================================================
    # TWO-STAGE CLASSIFIER
    # ============================================================================

    class TwoStageClassifier:
        """Two-stage news classifier: Real/Fake → Human/AI"""

        def __init__(self, model_s1, model_s2, tokenizer_s1, tokenizer_s2, device):
            self.model_s1 = model_s1
            self.model_s2 = model_s2
            self.tokenizer_s1 = tokenizer_s1
            self.tokenizer_s2 = tokenizer_s2
            self.device = device
            self.model_s1.eval()
            self.model_s2.eval()

        def predict(self, text, max_length=256):
            # Stage 1: Real/Fake
            encoding_s1 = self.tokenizer_s1.encode_plus(
                text, add_special_tokens=True, max_length=max_length,
                padding='max_length', truncation=True,
                return_attention_mask=True, return_tensors='pt'
            )

            input_ids_s1 = encoding_s1['input_ids'].to(self.device)
            attention_mask_s1 = encoding_s1['attention_mask'].to(self.device)

            with torch.no_grad():
                outputs_s1 = self.model_s1(input_ids=input_ids_s1, attention_mask=attention_mask_s1)
                probs_s1 = torch.softmax(outputs_s1.logits, dim=1)
                conf_s1, pred_s1 = torch.max(probs_s1, dim=1)

            is_real = pred_s1.item()
            real_confidence = conf_s1.item()

            result = {
                'is_real': is_real,
                'real_confidence': real_confidence,
                'real_label': 'Real' if is_real == 1 else 'Fake',
                'is_ai': None,
                'ai_confidence': None,
                'ai_label': 'N/A'
            }

            # Stage 2: Human/AI (only if Fake)
            if is_real == 0:
                encoding_s2 = self.tokenizer_s2.encode_plus(
                    text, add_special_tokens=True, max_length=max_length,
                    padding='max_length', truncation=True,
                    return_attention_mask=True, return_tensors='pt'
                )

                input_ids_s2 = encoding_s2['input_ids'].to(self.device)
                attention_mask_s2 = encoding_s2['attention_mask'].to(self.device)

                with torch.no_grad():
                    outputs_s2 = self.model_s2(input_ids=input_ids_s2, attention_mask=attention_mask_s2)
                    probs_s2 = torch.softmax(outputs_s2.logits, dim=1)
                    conf_s2, pred_s2 = torch.max(probs_s2, dim=1)

                is_ai = pred_s2.item()
                ai_confidence = conf_s2.item()

                result['is_ai'] = is_ai
                result['ai_confidence'] = ai_confidence
                result['ai_label'] = 'AI-generated' if is_ai == 1 else 'Human-written'

            return result

        def predict_batch(self, texts):
            return [self.predict(text) for text in texts]

    # Initialize classifier
    two_stage_classifier = TwoStageClassifier(
        model_s1, model_s2, tokenizer_s1, tokenizer_s2, device
    )

    print("\n✅ Two-Stage Classifier ready!")

# ============================================================================
# DEMO (works for both Stage 1 only and full two-stage)
# ============================================================================

print("\n" + "="*80)
print("DEMO: Two-Stage Classification")
print("="*80)

# Test on 3 examples
for i in range(min(3, len(X_test_s1))):
    text = X_test_s1[i]
    result = two_stage_classifier.predict(text)

    print(f"\nExample {i+1}:")
    print(f"Text: {text[:150]}...")
    print(f"Stage 1: {result['real_label']} ({result['real_confidence']:.2%})")
    if result['is_ai'] is not None:
        print(f"Stage 2: {result['ai_label']} ({result['ai_confidence']:.2%})")

# Quick evaluation on subset
test_subset = X_test_s1[:20]
labels_subset = y_test_s1[:20]

results = two_stage_classifier.predict_batch(test_subset)
correct = sum(1 for i, r in enumerate(results) if r['is_real'] == labels_subset[i])

print(f"\n✅ Test Accuracy (20 samples): {correct/len(test_subset):.2%}")
print("\n✅ Part 2 Complete!")

##### Stage 2 (Human/AI Classifier) Phase Summary

This stage was designed to further classify news articles identified as 'Fake' by Stage 1, determining if they were human-written or AI-generated. The approach depended on the availability of sufficient labeled data:

*   **Purpose:** The primary goal of Stage 2 was to classify news as 'Human-written' (0) or 'AI-generated' (1), but only for articles that had already been classified as 'Fake' by the Stage 1 model.
*   **Data Preparation:** The data for Stage 2 (df_stage2) was meticulously filtered from the combined_news DataFrame to include only samples with valid is_ai labels (i.e., 0.0 or 1.0), explicitly excluding the -1.0 (Not applicable) entries. This ensured that only relevant data for AI/human classification was used.
*   **Conditional Training:** Due to the sample size setting (SAMPLE_SIZE = 5000), the filtered df_stage2 contained only 142 samples (89 human-written, 53 AI-generated). While this is a small dataset for training a complex BERT model, the training process proceeded to demonstrate the full two-stage pipeline.
*   **Training Process (if applicable):** Similar to Stage 1, DistilBertTokenizer and DistilBertForSequenceClassification were used. The data was split into training (99 samples), validation (21 samples), and test (22 samples) sets, stratified to maintain class distribution. The model was trained for 2 EPOCHS using AdamW and a linear scheduler, with BATCH_SIZE = 8 and ACCUMULATION_STEPS = 2.
*   **Performance on Test Set:**
    *   **Test Accuracy:** The Stage 2 model achieved a test accuracy of 0.6364 (63.64%).
    *   **Classification Report:** The classification report showed a precision of 0.64 for the 'Human' class, but 0.00 for the 'AI' class. This indicates that while the model correctly identified most human-written examples, it struggled significantly with identifying AI-generated content, often misclassifying it as human-written. This is reflected in the recall for 'Human' being 1.00 (all human examples were predicted as human) and for 'AI' being 0.00 (no AI examples were predicted as AI).
    *   **Confusion Matrix:** The confusion matrix visually confirmed these results, showing that all 14 true human-written samples were correctly classified, but all 8 true AI-generated samples were incorrectly classified as human-written. This highlights a significant limitation likely due to the small size and potential imbalance of the Stage 2 training data.
*   **Two-Stage Classifier Integration:** Both trained models (Stage 1 and Stage 2) were integrated into a TwoStageClassifier. This classifier first uses model_s1 to predict if an article is real or fake. If it's classified as fake, model_s2 is then used to determine if it's human-written or AI-generated. This hierarchical structure allows for specialized classification at each step.


In [ ]:
# ============================================================================
# ATTENTION VISUALIZATION AND EXPLAINABILITY
# ============================================================================

print("\n" + "="*80)
print("ATTENTION VISUALIZATION & EXPLAINABILITY")
print("="*80)

from transformers import DistilBertModel

# ============================================================================
# ATTENTION EXTRACTION
# ============================================================================

class DistilBertWithAttention:
    """Extract attention weights from DistilBERT"""

    def __init__(self, tokenizer, device):
        self.tokenizer = tokenizer
        self.device = device
        self.model = DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.model.to(device)
        self.model.eval()

    def get_attention(self, text, max_length=256):
        """Extract attention weights and tokens"""
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        input_ids = encoding['input_ids'].to(self.device)
        attention_mask = encoding['attention_mask'].to(self.device)
        tokens = self.tokenizer.convert_ids_to_tokens(input_ids[0])

        with torch.no_grad():
            outputs = self.model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                output_attentions=True
            )

        return outputs.attentions, tokens

# Initialize for Stage 1
attention_viz_s1 = DistilBertWithAttention(tokenizer_s1, device)
print("✓ Attention visualizer ready")

# ============================================================================
# VISUALIZATION FUNCTIONS
# ============================================================================

def plot_attention_heatmap(attention, tokens, layer=0, head=0, max_tokens=40):
    """Plot attention heatmap for specific layer/head"""
    attn_matrix = attention[layer][0, head].cpu().numpy()

    # Get non-padding tokens
    non_pad_idx = min(max_tokens, len([t for t in tokens if t != '[PAD]']))
    attn_matrix = attn_matrix[:non_pad_idx, :non_pad_idx]
    tokens_display = tokens[:non_pad_idx]

    plt.figure(figsize=(10, 8))
    sns.heatmap(
        attn_matrix,
        xticklabels=tokens_display,
        yticklabels=tokens_display,
        cmap='viridis',
        cbar=True
    )
    plt.title(f'Attention Heatmap - Layer {layer}, Head {head}')
    plt.xlabel('Key Tokens')
    plt.ylabel('Query Tokens')
    plt.xticks(rotation=90)
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()

def get_token_importance(attention, tokens):
    """Calculate importance score for each token"""
    all_attentions = torch.stack([attn[0] for attn in attention])
    importance = all_attentions.sum(dim=(0, 1, 2)).cpu().numpy()

    token_importance = {}
    for i, token in enumerate(tokens):
        if token not in ['[CLS]', '[SEP]', '[PAD]']:
            token_importance[token] = importance[i]

    return token_importance

def plot_token_importance(token_importance, top_n=15, title="Token Importance"):
    """Plot top N most important tokens"""
    sorted_tokens = sorted(token_importance.items(), key=lambda x: x[1], reverse=True)
    top_tokens = sorted_tokens[:top_n]

    tokens = [t[0] for t in top_tokens]
    scores = [t[1] for t in top_tokens]

    plt.figure(figsize=(10, 6))
    plt.barh(range(len(tokens)), scores, color='steelblue')
    plt.yticks(range(len(tokens)), tokens)
    plt.xlabel('Importance Score')
    plt.title(title)
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

def explain_prediction(text, stage='stage1'):
    """Complete explainability analysis"""
    print("\n" + "="*70)
    print(f"EXPLAINING PREDICTION - {stage.upper()}")
    print("="*70)

    attention_viz = attention_viz_s1

    print(f"\nText: {text[:200]}...")

    # Get prediction
    result = two_stage_classifier.predict(text)
    print(f"\nPrediction: {result['real_label']}")
    print(f"Confidence: {result['real_confidence']:.2%}")

    # Get attention
    attention, tokens = attention_viz.get_attention(text)

    # Visualizations
    print("\n📊 Attention Heatmap (Layer 0, Head 0):")
    plot_attention_heatmap(attention, tokens, layer=0, head=0)

    print("\n📊 Most Important Tokens:")
    token_importance = get_token_importance(attention, tokens)
    plot_token_importance(token_importance, top_n=15, title="Top Important Tokens")

    return {
        'attention': attention,
        'tokens': tokens,
        'prediction': result,
        'token_importance': token_importance
    }

# ============================================================================
# DEMO: EXPLAIN PREDICTIONS
# ============================================================================

print("\n" + "="*80)
print("DEMO: EXPLAINING MODEL DECISIONS")
print("="*80)

# Analyze 2 examples from test set
for idx in [0, 5]:
    if idx < len(X_test_s1):
        example_text = X_test_s1[idx]
        print(f"\n{'='*80}")
        print(f"EXAMPLE {idx + 1}")
        results = explain_prediction(example_text, stage='stage1')

print("\n✅ Explainability analysis complete!")
print("\nYou can now use: explain_prediction(your_text, stage='stage1')")

# ============================================================================
# SUMMARY STATISTICS
# ============================================================================

print("\n" + "="*80)
print("PROJECT SUMMARY")
print("="*80)

print(f"\n📊 Stage 1 (Real/Fake Classifier):")
print(f"   Test Accuracy: {test_acc_s1:.2%}")
print(f"   Training Samples: {len(X_train_s1)}")

if 'test_acc_s2' in locals():
    print(f"\n📊 Stage 2 (Human/AI Classifier):")
    print(f"   Test Accuracy: {test_acc_s2:.2%}")
    print(f"   Training Samples: {len(X_train_s2)}")
else:
    print(f"\n📊 Stage 2: Not trained (no AI/Human labeled data)")

print(f"\n✅ Models saved:")
print(f"   - model_stage1_real_fake.pth")
if 'model_s2' in locals():
    print(f"   - model_stage2_human_ai.pth")

print("\n" + "="*80)
print("🎉 PROJECT COMPLETE!")
print("="*80)

##### Attention Visualization and Explainability Phase Summary

This phase was critical for understanding *how* the DistilBERT model arrived at its predictions, rather than just *what* its predictions were. By peering into the model's internal workings, we gained insights into which parts of the text were most influential in its decision-making process. The steps involved were:

*   **Attention Extraction:** A custom DistilBertWithAttention class was created to facilitate the extraction of attention weights directly from the DistilBertModel (a base DistilBERT model, not the fine-tuned classifier). This class tokenized the input text and then performed a forward pass through the model with output_attentions=True to retrieve the attention matrices for all layers and heads.
*   **Visualization Functions:**
    *   **Attention Heatmaps:** The plot_attention_heatmap function was developed to visualize the attention patterns within specific layers and heads of the model. These heatmaps illustrate how each token in a sequence attends to every other token, providing a visual representation of token-level relationships the model learned. For example, a heatmap would show if the model strongly associated certain keywords with the overall classification.
    *   **Token Importance Plots:** The get_token_importance function aggregated attention weights across all layers and heads to calculate an overall 'importance score' for each token. Tokens with higher scores were deemed more critical to the model's understanding of the text. The plot_token_importance function then visualized the top N most important tokens using a bar chart, clearly highlighting the words that caught the model's attention the most.
*   **Explain Prediction Function:** An explain_prediction function was implemented to integrate these capabilities. Given a text input, it first generated a prediction using the two-stage classifier, and then used the attention extraction and visualization functions to display the attention heatmap (for a specific layer/head) and the most important tokens. This provided a comprehensive explanation of the model's reasoning for a given input.
*   **Insights from Visualizations:** The attention visualizations provided concrete examples of words and phrases that significantly influenced the model's predictions. For instance, in real news articles, journalistic elements like city names (e.g., 'brussels', 'jerusalem') and news agency mentions (e.g., 'reuters') might show high importance, aligning with their factual nature. In fake news, highly charged emotional words or specific political figures' names could register as more important, reflecting the potentially sensational or opinionated content. These visual insights corroborated the findings from the EDA phase, particularly regarding word frequency and sentiment subjectivity, demonstrating how the model leverages these linguistic features.

##### Summary Analysis and Conclusions

This project successfully developed a two-stage classification system using DistilBERT to distinguish between real/fake and human/AI-generated news. The phased approach allowed for specialized model training and clearer insights into different aspects of news authenticity and authorship.

**Overall Assessment:**
*   **Stage 1 (Real/Fake Classifier):** Achieved outstanding performance with a test accuracy of **99.33%**. This model is highly effective at identifying whether a news article is real or fake, making it a robust initial filter.
*   **Stage 2 (Human/AI Classifier):** Demonstrated limited success with a test accuracy of **63.64%**, primarily due to its inability to correctly classify AI-generated content. The classification report and confusion matrix clearly showed that while the model performed well for human-written content (perfect recall for the Human class), it entirely missed AI-generated samples (zero recall for the AI class).

**Strengths and Limitations:**
*   **Strength - Stage 1:** The Real/Fake classifier is a significant success, proving that DistilBERT, even with sampled data, can generalize well to this task. The high F1-scores across both classes indicate balanced and reliable performance.
*   **Limitation - Stage 2:** The Human/AI classifiers poor performance for AI-generated content is a major limitation. This is likely attributable to the extremely small size of the AI-labeled dataset (only 142 samples, leading to just 22 test samples), which is insufficient for effective fine-tuning of a complex model like DistilBERT. The imbalance within this small dataset (89 human, 53 AI) further exacerbated the issue.

**Alignment with EDA and Explainability:**
*   **EDA Insights:** The EDA highlighted key differences: fake news was slightly longer and significantly more subjective than real news. While Stage 1s high accuracy suggests it leveraged these and other subtle linguistic cues effectively, the direct impact of these specific features on the models decision-making was further elucidated by attention visualization.
*   **Explainability:** The attention heatmaps and token importance plots provided valuable insights. They confirmed that the model focused on critical keywords, journalistic markers (like reuters in real news), and emotionally charged terms (potentially in fake news). This explainability layer helps build trust in the models predictions for Stage 1.

**Challenges and Future Improvements:**
1.  **Data Scarcity for Stage 2:** The primary challenge was the limited availability of high-quality, diverse AI-generated news data. A larger, more balanced dataset for distinguishing human from AI-generated text would drastically improve Stage 2s performance.
2.  **Model Complexity vs. Data Size:** Fine-tuning DistilBERT on only 142 samples for Stage 2 is suboptimal. Future work could explore:
    *   **Data Augmentation:** Techniques to generate more AI-labeled text.
    *   **Transfer Learning:** Freezing more layers of DistilBERT for Stage 2 if data remains scarce.
    *   **Simpler Models:** Using less complex models for Stage 2 if the feature space is very distinct.
3.  **Refining Explainability:** While attention is a good start, more advanced explainability techniques (e.g., LIME, SHAP) could offer deeper insights into feature contributions beyond just token importance.
4.  **Real-world Deployment:** For deployment, consider optimizing the two-stage classifier for inference speed and handling edge cases where a text might fall ambiguously between categories.

In conclusion, the project successfully built a highly accurate real/fake news detection system (Stage 1). However, the human/AI classification (Stage 2) requires substantial data augmentation or a re-evaluation of its modeling approach to become equally effective. The combination of strong predictive models and interpretability tools offers a promising framework for combating misinformation.

#### Baseline Logistic Regression

In [ ]:
import kagglehub #import module
import pandas as pd #import module
import os #import module
import re #import module
import numpy as np #import module

from sklearn.model_selection import train_test_split, GridSearchCV #import module
from sklearn.pipeline import Pipeline #import module
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer #import module
from sklearn.linear_model import LogisticRegression #import module
from sklearn.metrics import classification_report, ConfusionMatrixDisplay #import module
import matplotlib.pyplot as plt #import module

plt.rcParams["figure.figsize"] = (10, 5) #sets parameters

#download dataset
path_fake_real = kagglehub.dataset_download(
    "clmentbisaillon/fake-and-real-news-dataset"
)

#shows where dataset was downloaded in folder
print("fake/real path:", path_fake_real)
print("Contents of fake/real path:", os.listdir(path_fake_real))

#download dataset
path_ai1 = kagglehub.dataset_download(
    "walidbenaouda/ai-isot-dataset"
)

#shows where dataset was downloaded in folder
print("ai1 path:", path_ai1)
print("Contents of ai1 path:", os.listdir(path_ai1))

#download dataset
path_ai2 = kagglehub.dataset_download(
    "atharvasoundankar/gen-ai-misinformation-detection-datase-20242025"
)

#shows where dataset was downloaded in folder
print("ai2 path:", path_ai2)
print("Contents of ai2 path:", os.listdir(path_ai2))

fake = pd.read_csv(os.path.join(path_fake_real, "Fake.csv")) #cvs for fake
true = pd.read_csv(os.path.join(path_fake_real, "True.csv")) #cvs for true

#cvs for Ai misinformation
ai = pd.read_csv(os.path.join(
    path_ai2,
    "generative_ai_misinformation_dataset.csv"
))

print(fake.head()) #shows headers for fake
print(true.head()) #shows headers for true
print(ai.head()) #shows headers for AI misinformation

#cleans text in dataset
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r"http\S+|www.\S+", "", text) #gets rid of urls
    text = re.sub(r"<.*?>", "", text) #gets rid of html
    text = re.sub(r"[^a-zA-Z0-9.,!?'’\s]", " ", text) #makes sure punctuation still there
    text = re.sub(r"\s+", " ", text).strip() #fixes spaces
    return text

fake["label"] = 0 #gives fake news a label
true["label"] = 1 #gives true news a label

df_rf = pd.concat([fake, true], ignore_index=True) #combines datasets

#makes text in one place
df_rf["text"] = (
    df_rf["title"].fillna("") + " " +
    df_rf["text"].fillna("")
).str.strip()

#gets rid of duplicates
#gets rid of empty text
df_rf = df_rf.drop_duplicates(subset=["text"])
df_rf = df_rf.dropna(subset=["text"])
df_rf = df_rf[df_rf["text"].str.strip() != ""]

#cleans the text
df_rf["text"] = df_rf["text"].apply(clean_text)

#picks the necessary columns
df_rf_clean = df_rf[["text", "label"]]

#saves the cleaned file
df_rf_clean.to_csv("clean_real_fake.csv", index=False)

#shows it was saved
print("saved: clean_real_fake.csv")

#shows headers of cleaned dataset
df_rf_clean.head()

#picks the necessary columns
ai = ai[["text", "is_misinformation", "timestamp", "country"]].copy()

#renames columns
ai = ai.rename(columns={"is_misinformation": "label"})

#gets rid of duplicates
#gets rid of empty text
ai = ai.drop_duplicates(subset=["text"])
ai = ai.dropna(subset=["text"])
ai = ai[ai["text"].str.strip() != ""]

#cleans the text
ai["text"] = ai["text"].apply(clean_text)

#saves the cleaned file
ai.to_csv("clean_ai_human.csv", index=False)

#shows it was saved
print("saved: clean_ai_human.csv")

#shows headers of cleaned dataset
ai.head()

#uses copy of cleaned dataset for training
df = df_rf_clean.copy()

X = df["text"] #labels features from article text
y = df["label"] #labels target for fake and real

#splits into the sets for train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=.2,
    random_state=42,
    stratify=y
)

#changes text to tf-idf & logistic regression
tfidf_log_reg_pipeline = Pipeline([
    ("vec", TfidfVectorizer(
        stop_words="english",
        max_df=.9
    )),
    ("clf", LogisticRegression(
        max_iter=500,
        n_jobs=-1
    ))
])

#setting parameters for what we are searching for
tfidf_param_grid = {
    "vec__ngram_range": [(1, 1), (1, 2)],
    "clf__C": [.01, 1]
}

#how the search actually works
tfidf_grid = GridSearchCV(
    tfidf_log_reg_pipeline,
    param_grid=tfidf_param_grid,
    cv=3,
    scoring="f1_macro",
    n_jobs=1,
    verbose=2
)

#runs the gridsearch
tfidf_grid.fit(X_train, y_train)

#shows best hyperparameters
print("\n=== tf-idf + logistic regression ===")
print("best tf-idf params:", tfidf_grid.best_params_)
print("best cv score:", tfidf_grid.best_score_)

#sets prediction labels for test set
y_pred_tfidf = tfidf_grid.predict(X_test)
print("\ntf-idf logistic regression - test set report")
print(classification_report(y_test, y_pred_tfidf))

#makes confusion matrix plot
ConfusionMatrixDisplay.from_predictions(y_test, y_pred_tfidf)
plt.title("tf-idf + logistic regression - confusion matrix")
plt.show()

#bow & logisic regression
bow_log_reg_pipeline = Pipeline([
    ("vec", CountVectorizer(
        stop_words="english",
        max_df=.9
    )),
    ("clf", LogisticRegression(
        max_iter=500,
        n_jobs=-1
    ))
])

#setting parameters for what we are searching for
bow_param_grid = {
    "vec__ngram_range": [(1, 1)],
    "vec__min_df": [2],
    "clf__C": [.01],
    "clf__class_weight": [None, "balanced"]
}

#how the search actually works
bow_grid = GridSearchCV(
    bow_log_reg_pipeline,
    param_grid=bow_param_grid,
    cv=3,
    scoring="f1_macro",
    n_jobs=1,
    verbose=2
)

#runs gridsearch for bow model
bow_grid.fit(X_train, y_train)

#shows best hyperparameters
print("\n=== bow + logistic regression ===")
print("best bow params:", bow_grid.best_params_)
print("best cv score:", bow_grid.best_score_)

#sets prediction labels for test set
y_pred_bow = bow_grid.predict(X_test)
print("\nbow logistic regression - test set report")
print(classification_report(y_test, y_pred_bow))

#shows confusion matrix graph
ConfusionMatrixDisplay.from_predictions(y_test, y_pred_bow)
plt.title("bow + logistic regression - confusion matrix")
plt.show()

#gets date & label columns in real/fake dataset
df_trend = df_rf[["date", "label"]].copy()

#gets rid of duplicates
#gets rid of empty text
df_trend["date"] = pd.to_datetime(
    df_trend["date"],
    errors="coerce",
    dayfirst=True
)

#gets rid of rows where date isn't readable
df_trend = df_trend.dropna(subset=["date"])

#groups dates by first of month
df_trend["month"] = df_trend["date"].dt.to_period("M").dt.to_timestamp()

#shows headers and columns
print(df_trend.head())
print(df_trend.columns)

#groups by month & labels
trend_over_time = (
    df_trend.groupby(["month", "label"])   # <-- FIX
        .size()
        .reset_index(name="count")
)

#shows headers
print(trend_over_time.head())

#makes linechart for news count over time
plt.figure(figsize=(10, 5))
for lab in trend_over_time["label"].unique():
    subset = trend_over_time[trend_over_time["label"] == lab]
    plt.plot(subset["month"], subset["count"], marker="o", label=str(lab))

plt.title("news count over time") #titles chart
plt.xlabel("month") #labels xaxis
plt.ylabel("count") #labels yaxis
plt.legend() #makes legend
plt.xticks(rotation=45) #adjusts month labels
plt.tight_layout() #adjusts layout
plt.show() #shows linechart

#makes ai_trend with labels from cleaned dataset
ai_trend = ai[["timestamp", "country", "label"]].copy()

#gets timestamps
ai_trend["timestamp"] = pd.to_datetime(ai_trend["timestamp"])

#gets dates
ai_trend["date"] = ai_trend["timestamp"].dt.date

#changes back to datetime
ai_trend["date"] = pd.to_datetime(ai_trend["date"])

#makes groupings by month
ai_trend["month"] = ai_trend["date"].dt.to_period("M").dt.to_timestamp()

#sets patameters
def is_us_country(x):
    if isinstance(x, str):
        x_upper = x.upper()
        return ("USA" in x_upper) or ("UNITED STATES" in x_upper) or (x_upper == "US")
    return False

#makes region as US & international
ai_trend["region"] = np.where(ai_trend["country"].apply(is_us_country),
                              "US", "International")

#makes region, month & labels
#finds number of posts
#counts the found total
region_trend = (
    ai_trend.groupby(["month", "region", "label"])
            .size()
            .reset_index(name="count")
)

#shows headers
print(region_trend.head())

#keeps only misinformation posts labeled
misinfo_region_trend = region_trend[region_trend["label"] == 1]

#shows headers
print(misinfo_region_trend.head())

#makes linechart for time series based on region
plt.figure(figsize=(10, 5))
for region in misinfo_region_trend["region"].unique():
  subset = misinfo_region_trend[misinfo_region_trend["region"] == region]
  plt.plot(subset["month"], subset["count"], marker="o", label=region)


plt.title("misinformation over time: US vs international") #titles chart
plt.xlabel("month") #labels xaxis
plt.ylabel("number of misinformation posts") #labels yaxis
plt.legend() #makes legend
plt.xticks(rotation=45) #adjusts month labels
plt.tight_layout() #adjusts layout
plt.show() #shows linegraph

#installs lime
!pip install lime

from lime.lime_text import LimeTextExplainer #import module

#gets best model from tf-idf from gridsearch
best_model = tfidf_grid.best_estimator_

#sort labels listed in training data
label_values = sorted(y_train.unique())

#change labels to strings for lime
class_names = [str(l) for l in label_values]

#shows class names for lime
print("class names for lime:", class_names)

#makes lime explainer
explainer = LimeTextExplainer(class_names=class_names)

#picks text example to explain
idx = 10

#text of data
text_instance = X_test.iloc[idx]

#true label
true_label = y_test.iloc[idx]

#shows text from article
#shows true label
#shows models predicted label
print("=== TEXT EXAMPLE ==")
print(text_instance)
print("\ntrue label:", true_label)
print("model prediction:", best_model.predict([text_instance])[0])

#text we want to explain
exp = explainer.explain_instance(
    text_instance,
    best_model.predict_proba,
    num_features=10,
    top_labels=1
)

#shos explanation with highlighted words
exp.show_in_notebook(text=True)

#saves file as html
exp.save_to_file("lime_example_1.html")

#shows file has been saved
print("saved lime explanation to lime_example_1.html")